## Fine-tune Llama 3.2 11b with prepared dataset

In this notebook, we will go over the steps to fine-tune llama 3.2 11b with the dataset that was prepared in the previous notebook for building your own OCR solution.

We will ensure that the dataset is first loaded to s3 before we define our hyperparameters that will be used within our SageMaker estimator.

### Upload the dataset to the S3

Given the dataset contains image, the uploading process will take a while depending on the size of examples you process

In [ ]:
import boto3
import os
from botocore.exceptions import ClientError

# Initialize the S3 client
s3 = boto3.client('s3')
bucket_name = 'genai-accelerate-2024'
subdirectory = 'llama-3-2-vision-folder'
train_data_location = f"s3://{bucket_name}/{subdirectory}"

# Files to upload (metadata.jsonl and images folder)
files_to_upload = [
    '/home/ec2-user/SageMaker/vision-workshop/dataset/metadata.jsonl',
    '/home/ec2-user/SageMaker/vision-workshop/dataset/images'
]

for file_path in files_to_upload:
    if os.path.isfile(file_path):
        # Upload single file
        file_name = os.path.basename(file_path)
        key_path = f"{subdirectory}/{file_name}"
        try:
            s3.upload_file(file_path, bucket_name, key_path)
            print(f"File {file_name} uploaded successfully to {key_path}.")
        except ClientError as e:
            print(f"Error uploading file {file_name}: {e}")
    elif os.path.isdir(file_path):
        # Upload directory
        for root, _, files in os.walk(file_path):
            for file in files:
                full_file_path = os.path.join(root, file)
                relative_path = os.path.relpath(full_file_path, start=file_path)
                s3_key = f"{subdirectory}/images/{relative_path}"
                try:
                    s3.upload_file(full_file_path, bucket_name, s3_key)
                    print(f"File {file} uploaded successfully to {s3_key}.")
                except ClientError as e:
                    print(f"Error uploading file {file} to {s3_key}: {e}")

## Define model id and hyperparameters

In the next cell we will use the sagemaker library to call the model that we will fine-tune along with the hyperparameters. By default, we will choose 1 epoch as our hyperparamter for fine-tuning.

In [ ]:
from sagemaker import hyperparameters

# Set model ID and version
model_id, model_version = "meta-vlm-llama-3-2-11b-vision-instruct", "*"

my_hyperparameters = hyperparameters.retrieve_default(
    model_id=model_id, model_version=model_version
)
print(my_hyperparameters)

In [ ]:
my_hyperparameters["epoch"] = "1"

In [ ]:
hyperparameters.validate(
    model_id=model_id, model_version=model_version, hyperparameters=my_hyperparameters
)

In [ ]:
from sagemaker.jumpstart.estimator import JumpStartEstimator


estimator = JumpStartEstimator(
    model_id=model_id,
    model_version=model_version,
    environment={"accept_eula": "true"},  # Please change {"accept_eula": "true"}
    disable_output_compression=True,
    instance_type="ml.p4de.24xlarge",
    hyperparameters=my_hyperparameters,
)
estimator.fit({"training": train_data_location})

Studio Kernel Dying issue: If your studio kernel dies and you lose reference to the estimator object, please see section 2. Studio Kernel Dead/Creating JumpStart Model from the training Job on how to deploy endpoint using the training job name and the model id.

### Deploy the fine-tuned model

------

Next, we deploy fine-tuned model. We will compare the performance of fine-tuned and pre-trained model.

------
Before deployment, we need to know whether the model is finetuned with chat template. If so, we need deploy it with MESSAGES_API_ENABLED and input / output signatures are different.



In [1]:
is_chat_template = True if my_hyperparameters["chat_template"] == "True" else False

NameError: name 'my_hyperparameters' is not defined

In [ ]:
if is_chat_template:
    estimator.environment = {"MESSAGES_API_ENABLED": "true"}

In [ ]:
finetuned_predictor = estimator.deploy()